## Importy

In [21]:
import queue
from collections import defaultdict
from collections import deque
import numpy as np
from PIL import Image
from timeit import default_timer

## KMP

In [2]:
def prefix_function(pattern):
    pi = [0]
    k = 0
    for q in range(1, len(pattern)):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k - 1]
        if pattern[k] == pattern[q]:
            k = k + 1
        pi.append(k)
    return pi

In [3]:
def KMP_string_matching(text, pattern):
    pi = prefix_function(pattern)
    q = 0
    res = []
    for i in range(0, len(text)):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q - 1]
        if pattern[q] == text[i]:
            q = q + 1
        if q == len(pattern):
            res.append(i)
            q = pi[q - 1]
    return res

## Aho-Corasick

In [4]:
class Node:
    def __init__(self, c):
        self.fail: Node = None
        self.next = dict([])
        self.char = c
        self.output = []
        self.isRoot = 0

    def find_next(self, character):
        if character in self.next.keys():
            return self.next[character]
        elif self.isRoot:
            return self
        else:
            return None


class AhoCorasick:
    def __init__(self, patterns):
        self.patterns = patterns
        self.root = Node('')
        self.root.isRoot = 1
        self.transition_table()

    def transition_table(self):
        for pattern in self.patterns:
            current = self.root
            for character in pattern:
                if character not in current.next.keys():
                    current.next[character] = Node(character)
                current = current.next[character]
            current.output.append(pattern)
        self.create_fails()

    def create_fails(self):
        q = deque()
        for val in self.root.next.values():
            val.fail = self.root
            q.append(val)
        while q:
            current = q.popleft()
            for child in current.next.values():
                q.append(child)
                # if child.fail is not None
                node_fail = current.fail
                while node_fail.find_next(child.char) is None:
                    node_fail = node_fail.fail

                child.fail = node_fail.find_next(child.char)
                if child.fail:
                    child.output = child.output + child.fail.output

    def compute_text(self, text):
        current = self.root
        keywords_found = []
        current_state = 0
        for c in text:
            while current.find_next(c) is None:
                current = current.fail
            current = current.find_next(c)
            if current.output:
                for node in current.output:
                    keywords_found.append((current_state - len(node) + 1, node))
            current_state += 1
        return keywords_found

## Funkcje pomocnicze

In [12]:
def text_to_matrix(text):
    max_line_len = max([len(line) for line in text])
    res = []
    for row in text:
        line = list(row)
        for i in range(max_line_len - len(line)):
            line.append('\0')
        res.append(line)
    return res


def transpose_matrix(A):
    return list(map(list, zip(*A)))


def list_to_string(line):
    return str(np.array(line))


def matrix_to_KMP(text, patterns):
    # build matrix
    lines = text_to_matrix(text)
    A = AhoCorasick(patterns)
    patterns_found_matrix = [[0 for _ in range(len(lines[0]))] for _ in range(len(lines))]
    for i in range(len(lines)):
        line = lines[i]
        temp = A.compute_text(line)
        for elem in temp:
            patterns_found_matrix[i][elem[0]] = patterns.index(elem[1]) + 1
    res = transpose_matrix(patterns_found_matrix)
    return res

def png_to_matrix(path):
    png_array = np.array(Image.open(path))[:, :, 0]
    parsed_array = np.where(png_array > 128, 0, 1)
    matrix = ["". join(str(row))[1:-1] for row in parsed_array]
    return matrix

## Zadanie 2

In [6]:
def zad2():
    with open('haystack.txt', 'r') as file:
        text = file.readlines()
    alphabet = set()
    for line in text:
        for letter in line:
            if letter != ' ':
                alphabet.add(letter)
    for letter in alphabet:
        T = matrix_to_KMP(text, [letter])
        counter = 0
        for line in T:
            temp = KMP_string_matching(list_to_string(line), '1 1')
            counter += len(temp)
        print("Litera:", letter, ", ilość wystąpień jedna pod drugą:", counter)

zad2()        

Litera: d , ilość wystąpień jedna pod drugą: 1
Litera: L , ilość wystąpień jedna pod drugą: 0
Litera: ) , ilość wystąpień jedna pod drugą: 0
Litera: x , ilość wystąpień jedna pod drugą: 1
Litera: G , ilość wystąpień jedna pod drugą: 0
Litera: - , ilość wystąpień jedna pod drugą: 0
Litera: i , ilość wystąpień jedna pod drugą: 12
Litera: 1 , ilość wystąpień jedna pod drugą: 0
Litera: g , ilość wystąpień jedna pod drugą: 0
Litera: 6 , ilość wystąpień jedna pod drugą: 0
Litera: 5 , ilość wystąpień jedna pod drugą: 0
Litera: 7 , ilość wystąpień jedna pod drugą: 0
Litera: h , ilość wystąpień jedna pod drugą: 3
Litera: " , ilość wystąpień jedna pod drugą: 0
Litera: V , ilość wystąpień jedna pod drugą: 0
Litera: X , ilość wystąpień jedna pod drugą: 0
Litera: c , ilość wystąpień jedna pod drugą: 6
Litera: I , ilość wystąpień jedna pod drugą: 0
Litera: l , ilość wystąpień jedna pod drugą: 5
Litera: w , ilość wystąpień jedna pod drugą: 2
Litera: j , ilość wystąpień jedna pod drugą: 0
Litera: Q , 

## Zadanie 3

In [10]:
def zad3():
    with open('haystack.txt', 'r') as file:
        text = file.readlines()
    T1 = matrix_to_KMP(text, ['t h', 't h'])
    T2 = matrix_to_KMP(text, ['th', 'th'])
    sum1 = 0
    sum2 = 0
    print("'t h'")
    i = 0 
    for line in T1:
        temp = KMP_string_matching(list_to_string(line), '1 1')
        if len(temp) > 0:
            for elem in temp:
                print((elem-1)//2 - 1, i)
        i+=1        
        sum1 += len(temp)
    print("Ilość wystąpień 't h' jedna pod drugą:", sum1)
    print("'th'")
    i = 0
    for line in T2:
        temp = KMP_string_matching(list_to_string(line), '1 1')
        i+=1
        sum2 += len(temp)
    print("Ilość wystąpień 'th' jedna pod drugą:", sum2)
    
zad3()    

't h'
37 0
Ilość wystąpień 't h' jedna pod drugą: 1
'th'
Ilość wystąpień 'th' jedna pod drugą: 0


## Zadanie 4

In [19]:
def zad4():
    text = png_to_matrix('haystack.png')
    pattern1 = png_to_matrix('literkaa.png')
    pattern2 = png_to_matrix('literkao.png')
    pattern3 = png_to_matrix('literkag.png')
    patterns = [pattern1, pattern2, pattern3]
    letters = ['a', 'o', 'g']
    sumy = 3*[0]
    i = 0
    for pattern in patterns:
        suma = 0
        T = matrix_to_KMP([list_to_string(line) for line in text], [list_to_string(line) for line in pattern])
        for line in T:
            matching = KMP_string_matching(' '.join([str(i) for i in line]), ' '.join([str(pattern.index(line) + 1) for line in pattern]))
            suma += len(matching)
        print("Ilość wystąpień literki", letters[i], ":", suma)
        i += 1

zad4()

Ilość wystąpień literki a : 288
Ilość wystąpień literki o : 259
Ilość wystąpień literki g : 69


## Zadanie 5

In [20]:
def zad5():
    text = png_to_matrix('haystack.png')
    pattern1 = png_to_matrix('patternik.png')
    suma = 0
    T = matrix_to_KMP([list_to_string(line) for line in text], [list_to_string(line) for line in pattern1])
    for line in T:
        matching = KMP_string_matching(' '.join([str(i) for i in line]),
                                       ' '.join([str(pattern1.index(line) + 1) for line in pattern1]))
        suma += len(matching)
    print("Ilość dopasowań 'p a t t e r n':", suma)


zad5()

Ilość dopasowań 'p a t t e r n': 0


## Zadanie 6

In [24]:
def measure_matrix_to_KMP(text, patterns):
    lines = text_to_matrix(text)
    start_automata = default_timer()
    A = AhoCorasick(patterns)
    end_automata = default_timer()
    patterns_found_matrix = [[0 for _ in range(len(lines[0]))] for _ in range(len(lines))]
    start_matching = default_timer()
    for i in range(len(lines)):
        line = lines[i]
        temp = A.compute_text(line)
        for elem in temp:
            patterns_found_matrix[i][elem[0]] = patterns.index(elem[1]) + 1
    end_matching = default_timer()        
    res = transpose_matrix(patterns_found_matrix)
    return res, (end_automata - start_automata), (end_matching - start_matching)

In [27]:
def zad6():
    text = png_to_matrix('haystack.png')
    pattern1 = png_to_matrix('literkaa.png')
    pattern2 = png_to_matrix('patternik.png')
    pattern3 = png_to_matrix('paternduzy.png')
    patterns = [pattern1, pattern2, pattern3]
    sizes = ['mały', 'średni', 'duży']
    i = 0
    for pattern in patterns:
        T, automata_time, matching_time = measure_matrix_to_KMP([list_to_string(line) for line in text],
                                                                [list_to_string(line) for line in pattern])
        print("Dla wzorca wielkości: ", sizes[i], ",czas tworzenia to:", automata_time, "czas dopasowania:",
              matching_time)
        i += 1


zad6()

Dla wzorca wielkości:  mały ,czas tworzenia to: 0.008788900000581634 czas dopasowania: 2.7120473999984824
Dla wzorca wielkości:  średni ,czas tworzenia to: 0.07001360000140266 czas dopasowania: 1.92305160000069
Dla wzorca wielkości:  duży ,czas tworzenia to: 0.4727167000000918 czas dopasowania: 2.2470856999989337


## Zadanie 7

In [29]:
print("Dla pliku hastack.txt:")
def zad7():
    with open('haystack.txt', 'r') as file:
        text = file.readlines()
    parts = [2, 4, 8]
    for part in parts:
        len_of_part = len(text)//part
        for i in range(part):
            new_text = []
            for j in range(len_of_part):
                new_text.append(text[i*len_of_part + j])
            res0, res1, res2 = measure_matrix_to_KMP(new_text, ['t h'])
            print("Wyszukiwanie dzieląc na:", part, "części trwa:", res2)

            
zad7()

Dla pliku hastack.txt:
Wyszukiwanie dzieląc na: 2 części trwa: 0.002297200000612065
Wyszukiwanie dzieląc na: 2 części trwa: 0.0023770999996486353
Wyszukiwanie dzieląc na: 4 części trwa: 0.0009769000007509021
Wyszukiwanie dzieląc na: 4 części trwa: 0.0010241999989375472
Wyszukiwanie dzieląc na: 4 części trwa: 0.0011900000008608913
Wyszukiwanie dzieląc na: 4 części trwa: 0.000992999999652966
Wyszukiwanie dzieląc na: 8 części trwa: 0.0004809000001841923
Wyszukiwanie dzieląc na: 8 części trwa: 0.0007462000012310455
Wyszukiwanie dzieląc na: 8 części trwa: 0.0004929999995511025
Wyszukiwanie dzieląc na: 8 części trwa: 0.000480799999422743
Wyszukiwanie dzieląc na: 8 części trwa: 0.0006137999989732634
Wyszukiwanie dzieląc na: 8 części trwa: 0.0006579000000783708
Wyszukiwanie dzieląc na: 8 części trwa: 0.0004964999989169883
Wyszukiwanie dzieląc na: 8 części trwa: 0.00047540000014123507
